### 12-31-17

- #### Q: the result scale will be affected by the sentence length and iteration in training process ?


### 12-30-17

- start

- finish bigram model in english and spanish

In [1]:
import numpy as np
import pandas as pd
%pylab inline 
from scipy.stats import norm as gaussian 
from scipy.misc import logsumexp

Populating the interactive namespace from numpy and matplotlib


# sample

- Eng

A story in the Daily Mail said that just three per cent of people in the UK know the words to Auld Lang Syne, the song traditionally sung at the turn of the New Year.
Research by Sainsbury's shows the majority can sing the chorus and first few lines at best, but 42 per cent do not know a single word.
A spokesman for Sainsbury's commented : “We want everyone to have a great New Year's Eve and singing Auld Lang Syne - or Old Land Sign as some people thought - is as much a part of our celebrations as a glass of fizz at midnight.

- Span

Y es que aun cuando las bitcoins son uno se los temas más nombrados este final de ano y a en 10 horas se pueden reportar mas de 184 mil transacciones en el mundo, muchos pereiranos desconfían.
En esta ciudad donde está el primer cajero en Colombia por fuera de Bogota, algunos ciudadanos temen que, como con las piramides, su platica termine intervenida y perdida.
"La verdad es que no es confiable algo que no tiene el respaldo de una autoridad y por eso me abstengo de usarlas", dijo una de las ciudadanas consultadas.

In [129]:
eng_test = "A story in the Daily Mail said that just three per cent of people in the UK know the words to Auld Lang Syne, the song traditionally sung at the turn of the New Year. Research by Sainsbury's shows the majority can sing the chorus and first few lines at best, but 42 per cent do not know a single word. A spokesman for Sainsbury's commented : “We want everyone to have a great New Year's Eve and singing Auld Lang Syne - or Old Land Sign as some people thought - is as much a part of our celebrations as a glass of fizz at midnight. "

In [130]:
span_test = "Y es que aun cuando las bitcoins son uno se los temas más nombrados este final de ano y a en 10 horas se pueden reportar mas de 184 mil transacciones en el mundo, muchos pereiranos desconfían. En esta ciudad donde está el primer cajero en Colombia por fuera de Bogota, algunos ciudadanos temen que, como con las piramides, su platica termine intervenida y perdida. La verdad es que no es confiable algo que no tiene el respaldo de una autoridad y por eso me abstengo de usarlas, dijo una de las ciudadanas consultadas."

## Char index

In [156]:
ord("a".lower()) - 97

0

## Algorithm 

In [190]:
def loglinear_birgram(sentence, w = None, reg = None):
    
    i = 0
    maxIter = 40
    alpha, count = 0.1, 0
    
    if w == None:
        w1 = np.zeros(27)
        w2 = np.zeros((27,27))
    
    prev = 26
    
    while (i < maxIter) : # and abs(sum(w)) < predictors*5 :
        i += 1
           
        for char in sentence:
            if char.isalpha():
                cur = ord(char.lower())-97
            elif char == " ":
                cur = 26
            else:
                continue
            
            # get total probability
            total = sum(np.exp( w1 + w2[prev] ))
            #print total
            obs_w1 = np.array([0.0 if j != cur else 1 for j in xrange(27) ])
            obs_w2 = np.zeros((27,27))
            obs_w2[prev][cur] = 1.0
            
            w1 = w1 + alpha*( obs_w1 -  np.exp( w1 + np.sum(w2.T, axis = 0))/total )
            w2 = w2 + alpha*( obs_w2 -  np.exp( w1 + np.sum(w2.T, axis = 0))/total )
            
            alpha /= (1 + alpha**2) # arctangent convergence
            prev = cur
    
    return w1, w2
    

In [191]:
w1_e,w2_e = loglinear_birgram(eng_test) 
w1_e

array([ 14.32116704,   2.12093948,   2.87594885,   5.14897324,
        16.22412201,   2.85193702,   4.64388527,   6.72918852,
         8.14280278,   0.70230458,   1.44220489,   5.90392772,
         2.84514725,  12.88018249,  11.02061713,   2.91282632,
        -0.0831937 ,   8.87182977,  13.20536238,  13.44158131,
         4.77089236,   0.99183969,   3.64371171,  -0.0831937 ,
         4.47273999,   0.61203154,   9.20924086])

In [192]:
w1_s,w2_s = loglinear_birgram(span_test) 
w1_s

array([ 17.74139536,   2.21096285,   5.98338512,  11.06802838,
        19.70393126,   1.43969494,   1.37918408,   0.70189382,
         8.16995988,   0.64895677,  -0.08255073,   6.6555555 ,
         5.28057493,  13.55996617,  14.2544962 ,   3.66097735,
         1.43916422,   8.19063529,  13.57579479,   5.96039474,
         7.85505957,   0.64566923,  -0.08255073,  -0.08255073,
         1.46991026,  -0.08255073,  10.86246014])

In [193]:
test = "hi my name is lenny"

In [194]:
p1 = 1.0
prev = 26
for char in test:
    if char.isalpha():
        cur = ord(char.lower())-97
    elif char == " ":
        cur = 26
    else:
        continue


    # get total probability
    total = sum(np.exp( w1_e + w2_e[prev] ))
    
    p1 *= np.exp( w1_e[cur] + w2_e[prev][cur])/total 
    prev = cur
p1

1.1678599025378783e-117

In [195]:
p2 = 1.0
prev = 26
for char in test:
    if char.isalpha():
        cur = ord(char.lower())-97
    elif char == " ":
        cur = 26
    else:
        continue


    # get total probability
    total = sum(np.exp( w1_s + w2_s[prev] ))
    
    p2 *= np.exp( w1_s[cur] + w2_s[prev][cur])/total 
    prev = cur
p2

4.0252512127187546e-122

In [196]:
test = "hola mi nombre es Lenny"

In [197]:
p1, p2 = 1.0, 1.0
prev = 26
for char in test:
    if char.isalpha():
        cur = ord(char.lower())-97
    elif char == " ":
        cur = 26
    else:
        continue


    # get total probability
    total1 = sum(np.exp( w1_e + w2_e[prev] ))
    total2 = sum(np.exp( w1_s + w2_s[prev] ))
    
    p1 *= np.exp( w1_e[cur] + w2_e[prev][cur])/total1 
    p2 *= np.exp( w1_s[cur] + w2_s[prev][cur])/total2
    
    prev = cur
print "the probability that the sentence is English:", p1
print "the probability that the sentence is Spanish:", p2

the probability that the sentence is English: 6.84388825837e-126
the probability that the sentence is Spanish: 3.17736938522e-124


## Check probability

In [205]:
a = 0
for i in range(27):
    total2 = sum(np.exp( w1_s + w2_s[prev] ))
    a += np.exp( w1_s[i] + w2_s[prev][i])/total2
a

1.0000000000000004